In [19]:
import serial
import serial.tools.list_ports

print("pyserial OK")
print([p.device for p in serial.tools.list_ports.comports()])



pyserial OK
['COM6', 'COM10', 'COM7', 'COM12', 'COM4', 'COM5', 'COM8', 'COM9', 'COM13', 'COM22', 'COM3', 'COM23', 'COM16']


In [15]:
#This is just for testing don't use 
# import serial, time 

# COM_PORT="COM16"
# BAUD=115200

# with serial.Serial(COM_PORT, BAUD, timeout=2) as ser:
#     time.sleep(1.2)
#     ser.reset_input_buffer()
#     ser.write(b"STATUS\n")
#     t0=time.time()
#     while time.time()-t0 < 3:
#         line = ser.readline().decode(errors="ignore").strip()
#         if line:
#             print("ESP:", line)


In [1]:
import time, json, os
import serial

#User Input Information HANDSHAKE 
COM_PORT = "COM16"
BAUD     = 115200

SSID = "KGB_SURVEILLANCE_VAN"
PASS = "Watermelon123"
PORT = 9000 #Buggy!

CFG_FILE = "esp_last.json"

DO_PROVISION = True   # True = send WIFI ... ; False = just STATUS

def provision_get_ip(com_port: str, baud: int, ssid: str, password: str, port: int, do_provision: bool):
    with serial.Serial(com_port, baud, timeout=2) as ser:
        time.sleep(1.2)
        ser.reset_input_buffer()

        if do_provision:
            ser.write(f"WIFI ssid={ssid} pass={password} port={port}\n".encode())
        else:
            ser.write(b"STATUS\n")

        ip = None
        got_port = None
        t0 = time.time()

        while time.time() - t0 < 25:
            line = ser.readline().decode("utf-8", "ignore").strip()
            if not line:
                continue
            print("[usb]", line)

            # best: machine-readable line you added on ESP
            if "OK wifi_connected" in line:
                for p in line.split():
                    if p.startswith("ip="):   ip = p.split("=", 1)[1]
                    if p.startswith("port="): got_port = int(p.split("=", 1)[1])
                break

            # status response format: OK status ssid=... ip=... port=...
            if line.startswith("OK status"):
                for p in line.split():
                    if p.startswith("ip="):   ip = p.split("=", 1)[1]
                    if p.startswith("port="): got_port = int(p.split("=", 1)[1])
                if ip is not None and got_port is not None:
                    break

            # fallback prints
            if line.startswith("ESP32 IP:"):
                ip = line.split(":", 1)[1].strip()

            if "TCP server started on port" in line:
                try:
                    got_port = int(line.split("port", 1)[1].split(".", 1)[0].strip())
                except:
                    pass

            if ip is not None and got_port is not None:
                break

        if ip is None:
            raise RuntimeError("Did not get IP from ESP over USB serial")
        if got_port is None:
            got_port = port

        return ip, got_port

ESP_IP, ESP_PORT = provision_get_ip(COM_PORT, BAUD, SSID, PASS, PORT, DO_PROVISION)
print(f"[handshake] ESP_IP={ESP_IP} ESP_PORT={ESP_PORT}")

with open(CFG_FILE, "w") as f:
    json.dump({"ip": ESP_IP, "port": ESP_PORT}, f)
print(f"[handshake] saved {CFG_FILE}")


[usb] SD mount failed
[usb] Open /AndyAlphaWaveDataCSV.csv failed
[usb] # Provision over USB Serial:
[usb] # WIFI ssid=YOURSSID pass=YOURPASS port=9000
[usb] # Commands: STATUS, CLEAR
[usb] OK saved
[usb] ERR wifi_failed
[usb] # Auto-connect...
[usb] E (33350) wifi:sta is connecting, cannot set config


RuntimeError: Did not get IP from ESP over USB serial

In [26]:
import socket, time, json, os

RETRY_SEC = 1.5
CFG_FILE = "esp_last.json"

# If Cell 1 wasn't run in this kernel, load last saved IP/port
if "ESP_IP" not in globals() or "ESP_PORT" not in globals():
    if not os.path.exists(CFG_FILE):
        raise RuntimeError("No ESP_IP/ESP_PORT in memory and no esp_last.json. Run Cell 1 first.")
    with open(CFG_FILE, "r") as f:
        d = json.load(f)
    ESP_IP, ESP_PORT = d["ip"], int(d["port"])
    print(f"[stream] loaded ESP_IP={ESP_IP} ESP_PORT={ESP_PORT}")

def stream_samples(host: str, port: int):
    buf = b""
    while True:
        s = None
        try:
            s = socket.create_connection((host, port), timeout=5)
            s.settimeout(None)
            print(f"[tcp] connected {host}:{port}")
            buf = b""

            while True:
                chunk = s.recv(4096)
                if not chunk:
                    raise ConnectionError("server closed connection")
                buf += chunk
                while b"\n" in buf:
                    line, buf = buf.split(b"\n", 1)
                    line = line.strip()
                    if not line or line.startswith(b"#"):
                        continue
                    try:
                        t_ms_b, val_b = line.split(b",", 1)
                        yield int(t_ms_b), float(val_b)
                    except:
                        pass

        except KeyboardInterrupt:
            try:
                if s: s.shutdown(socket.SHUT_RDWR)
            except:
                pass
            if s: s.close()
            print("\n[bye]")
            raise

        except Exception as e:
            print(f"[tcp] reconnect: {e}; retrying in {RETRY_SEC}s…")
            try:
                if s: s.close()
            except:
                pass
            time.sleep(RETRY_SEC)

# ---- run stream ----
for t_ms, val in stream_samples(ESP_IP, ESP_PORT):
    print(f"{t_ms},{val}")


[tcp] connected 10.0.0.103:9000
6700,0.0
6704,12.44
6708,24.09
6712,34.23
6716,42.22
6720,47.55
6724,49.9
6728,49.11
6732,45.24
6736,38.52
6740,29.39
6744,18.4
6748,6.26
6752,-6.27
6756,-18.41
6760,-29.39
6764,-38.53
6768,-45.24
6772,-49.11
6776,-49.9
6780,-47.55
6784,-42.22
6788,-34.23
6792,-24.09
6796,-12.43
6800,0.0
6804,12.44
6808,24.09
6812,34.23
6816,42.22
6820,47.55
6824,49.9
6828,49.11
6832,45.24
6836,38.52
6840,29.39
6844,18.4
6848,6.26
6852,-6.27
6856,-18.41
6860,-29.39
6864,-38.53
6868,-45.24
6872,-49.11
6876,-49.9
6880,-47.55
6884,-42.22
6888,-34.23
6892,-24.09
6896,-12.43
6900,0.0
6904,12.44
6908,24.09
6912,34.23
6916,42.22
6920,47.55
6924,49.9
6928,49.11
6932,45.24
6936,38.52
6940,29.39
6944,18.41
6948,6.26
6952,-6.27
6956,-18.41
6960,-29.39
6964,-38.53
6968,-45.24
6972,-49.11
6976,-49.9
6980,-47.55
6984,-42.22
6988,-34.23
6992,-24.09
6996,-12.43
7000,0.0
7004,12.44
7008,24.09
7012,34.23
7016,42.22
7020,47.55
7024,49.9
7028,49.11
7032,45.24
7036,38.52
7040,29.39
7044,18.4


KeyboardInterrupt

